In [23]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
import matplotlib.pyplot as plt
import os
import pickle


from sklearn.linear_model import Ridge, RidgeCV
from sklearn.model_selection import cross_val_score

In [24]:
# --- Step 1: Load the datasets ---
print("="*60)
print("MACHINE LEARNING ASSIGNMENT - RIDGE REGRESSION")
print("="*60)

try:
    # Reading from the Hotel-Property-Value-Dataset folder
    train_df = pd.read_csv('../Hotel-Property-Value-Dataset/train.csv')
    test_df = pd.read_csv('../Hotel-Property-Value-Dataset/test.csv')
    sample_submission_df = pd.read_csv('../Hotel-Property-Value-Dataset/sample_submission.csv')
    print("✅ Files loaded successfully!")
    print(f"Training data shape: {train_df.shape}")
    print(f"Test data shape: {test_df.shape}")
except FileNotFoundError as e:
    print(f"❌ Error: {e}")
    print("Ensure the Hotel-Property-Value-Dataset folder contains train.csv, test.csv, and sample_submission.csv")
    exit()

# --- Target Variable ---
TARGET_VARIABLE = "HotelValue"

if TARGET_VARIABLE not in train_df.columns:
    print(f"❌ Error: The target column '{TARGET_VARIABLE}' was not found in train.csv.")
    print(f"Available columns are: {list(train_df.columns)}")
    exit()

print(f"\n📊 Target variable: {TARGET_VARIABLE}")
print(f"Target statistics:\n{train_df[TARGET_VARIABLE].describe()}")

MACHINE LEARNING ASSIGNMENT - RIDGE REGRESSION
✅ Files loaded successfully!
Training data shape: (1200, 81)
Test data shape: (260, 80)

📊 Target variable: HotelValue
Target statistics:
count      1200.000000
mean     181709.895833
std       77638.660223
min       34900.000000
25%      130000.000000
50%      165000.000000
75%      215000.000000
max      745000.000000
Name: HotelValue, dtype: float64


In [25]:
# --- Step 2: Data Preprocessing (Course Concepts) ---
print(f"\n" + "="*60)
print("STEP 2: DATA PREPROCESSING")
print("="*60)

# Separate features (X) from the target (y)
X_train_full = train_df.drop([TARGET_VARIABLE], axis=1)
y_train = train_df[TARGET_VARIABLE]
X_test_full = test_df.copy()

print(f"Features in training data: {X_train_full.shape[1]}")
print(f"Features in test data: {X_test_full.shape[1]}")

# Identify numeric and categorical columns
numeric_features = X_train_full.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X_train_full.select_dtypes(include=['object']).columns.tolist()

print(f"\n📈 Numeric features ({len(numeric_features)}): {numeric_features[:5]}..." if len(numeric_features) > 5 else f"\n📈 Numeric features ({len(numeric_features)}): {numeric_features}")
print(f"📝 Categorical features ({len(categorical_features)}): {categorical_features[:5]}..." if len(categorical_features) > 5 else f"📝 Categorical features ({len(categorical_features)}): {categorical_features}")


STEP 2: DATA PREPROCESSING
Features in training data: 80
Features in test data: 80

📈 Numeric features (37): ['Id', 'PropertyClass', 'RoadAccessLength', 'LandArea', 'OverallQuality']...
📝 Categorical features (43): ['ZoningCategory', 'RoadType', 'ServiceLaneType', 'PlotShape', 'LandElevation']...


In [26]:
# --- Step 3: Handle Missing Values and Encode Categorical Variables ---

# Configuration: Set your preferred cache directory here
CACHE_DIR = '../Hotel-Property-Value-Dataset/'  # Change this path as needed

def save_cleaned_data(X_train_processed, y_train, base_path=CACHE_DIR):
    """Save cleaned data as CSV file"""
    os.makedirs(base_path, exist_ok=True)
    
    # Combine features and target into single dataframe
    cleaned_df = X_train_processed.copy()
    cleaned_df['HotelValue'] = y_train
    
    # Save as CSV
    csv_path = os.path.join(base_path, 'train_cleaned_mada.csv')
    cleaned_df.to_csv(csv_path, index=False)
    
    print(f"✅ Cleaned data saved to {csv_path}")
    print(f"   Shape: {cleaned_df.shape}")
    return csv_path

def load_cleaned_data(base_path=CACHE_DIR):
    """Load cleaned data if it exists"""
    csv_path = os.path.join(base_path, 'train_cleaned_mada.csv')
    
    if os.path.exists(csv_path):
        cleaned_df = pd.read_csv(csv_path)
        
        # Separate features and target
        X_train_processed = cleaned_df.drop('HotelValue', axis=1)
        y_train = cleaned_df['HotelValue']
        
        print(f"✅ Cleaned data loaded from {csv_path}")
        print(f"   Shape: {cleaned_df.shape}")
        return X_train_processed, y_train, True
    else:
        print("❌ Cleaned data not found. Will perform preprocessing...")
        return None, None, False

def preprocess_data(X_train, X_test, numeric_features, categorical_features):
    """
    Preprocess the data by handling missing values and encoding categorical variables
    Following course preprocessing concepts
    """
    X_train_processed = X_train.copy()
    X_test_processed = X_test.copy()
    
    # Handle numeric features - Fill with median (robust to outliers)
    for col in numeric_features:
        if col in X_train_processed.columns:
            median_val = X_train_processed[col].median()
            X_train_processed[col] = X_train_processed[col].fillna(median_val)
            X_test_processed[col] = X_test_processed[col].fillna(median_val)
    
    # Handle categorical features - Label encoding
    label_encoders = {}
    for col in categorical_features:
        if col in X_train_processed.columns:
            # Fill missing values with mode (most frequent value)
            mode_val = X_train_processed[col].mode()[0] if not X_train_processed[col].mode().empty else 'Unknown'
            X_train_processed[col] = X_train_processed[col].fillna(mode_val)
            X_test_processed[col] = X_test_processed[col].fillna(mode_val)
            
            # Label encode categorical variables
            le = LabelEncoder()
            # Fit on combined data to handle unseen categories in test set
            combined_data = pd.concat([X_train_processed[col], X_test_processed[col]], axis=0)
            le.fit(combined_data)
            
            X_train_processed[col] = le.transform(X_train_processed[col])
            X_test_processed[col] = le.transform(X_test_processed[col])
            label_encoders[col] = le
    
    return X_train_processed, X_test_processed, label_encoders

# Try to load cleaned data first
X_train_processed, y_train_series, data_loaded = load_cleaned_data()

if not data_loaded:
    print("🔄 Starting data preprocessing...")
    
    # Preprocess the data
    X_train_processed, X_test_processed, label_encoders = preprocess_data(
        X_train_full, X_test_full, numeric_features, categorical_features
    )

    print(f"\nAfter preprocessing:")
    print(f"✅ Training data shape: {X_train_processed.shape}")
    print(f"✅ Test data shape: {X_test_processed.shape}")
    print(f"✅ Missing values in training data: {X_train_processed.isnull().sum().sum()}")
    print(f"✅ Missing values in test data: {X_test_processed.isnull().sum().sum()}")

    # Save cleaned training data as CSV
    y_train_for_saving = y_train.values if hasattr(y_train, 'values') else y_train
    save_cleaned_data(X_train_processed, y_train_for_saving)
    
    # Convert y_train to pandas Series for consistency
    y_train_series = pd.Series(y_train_for_saving, name='HotelValue')
    
else:
    print(f"✅ Using cleaned data from CSV!")
    print(f"Training features shape: {X_train_processed.shape}")
    print(f"Target shape: {y_train_series.shape}")
    
    # Still need to preprocess test data when loading from CSV
    print("🔄 Processing test data...")
    _, X_test_processed, label_encoders = preprocess_data(
        X_train_full, X_test_full, numeric_features, categorical_features
    )

# Feature Scaling (Standardization)
print(f"\n" + "="*60)
print("STEP 3: FEATURE SCALING")
print("="*60)

# Scale all features using standardization (mean=0, std=1)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_processed)
X_test_scaled = scaler.transform(X_test_processed)

print(f"✅ Features scaled successfully!")
print(f"Training features shape: {X_train_scaled.shape}")
print(f"Test features shape: {X_test_scaled.shape}")
print(f"Feature means after scaling: {np.mean(X_train_scaled, axis=0)[:5]} (should be ~0)")
print(f"Feature stds after scaling: {np.std(X_train_scaled, axis=0)[:5]} (should be ~1)")

# Convert to numpy array for mathematical operations
y_train = y_train_series.values


✅ Cleaned data loaded from ../Hotel-Property-Value-Dataset/train_cleaned_mada.csv
   Shape: (1200, 81)
✅ Using cleaned data from CSV!
Training features shape: (1200, 80)
Target shape: (1200,)
🔄 Processing test data...

STEP 3: FEATURE SCALING
✅ Features scaled successfully!
Training features shape: (1200, 80)
Test features shape: (260, 80)
Feature means after scaling: [-9.47390314e-17  1.77635684e-17  2.54611147e-16 -7.69754630e-17
 -6.21724894e-17] (should be ~0)
Feature stds after scaling: [1. 1. 1. 1. 1.] (should be ~1)


In [27]:
# === STEP 4: RIDGE REGRESSION USING SCIKIT-LEARN ===
# Using sklearn's Ridge for efficiency and built-in cross-validation

print(f"\n" + "="*60)
print("STEP 4: RIDGE REGRESSION - SCIKIT-LEARN IMPLEMENTATION")
print("="*60)


print("📐 Using sklearn Ridge Regression:")
print("Objective: Minimize ||Xθ - y||² + λ||θ||²")
print(f"Training data shape: {X_train_scaled.shape}")

# Ridge regression hyperparameter (regularization strength)
# Try different alpha values to find optimal regularization
alpha_values = [0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

print(f"\n🔍 HYPERPARAMETER TUNING - TESTING α VALUES:")
print("-" * 50)

# Use RidgeCV for automatic cross-validation
ridge_cv = RidgeCV(alphas=alpha_values, cv=5, scoring='r2')
ridge_cv.fit(X_train_scaled, y_train)

best_alpha = ridge_cv.alpha_
print(f"✅ Best α (regularization) found via CV: {best_alpha}")

# Train final model with best alpha
model = Ridge(alpha=best_alpha)
model.fit(X_train_scaled, y_train)

# Test different alpha values for comparison
all_results = {}
for alpha in alpha_values:
    ridge_temp = Ridge(alpha=alpha)
    ridge_temp.fit(X_train_scaled, y_train)
    r2_score = ridge_temp.score(X_train_scaled, y_train)
    
    # Calculate regularization penalty
    reg_penalty = alpha * np.sum(ridge_temp.coef_ ** 2)
    
    all_results[alpha] = {
        'model': ridge_temp,
        'r2': r2_score,
        'reg_penalty': reg_penalty,
        'coef_norm': np.linalg.norm(ridge_temp.coef_)
    }
    
    print(f"α = {alpha:8.2f} | R² = {r2_score:.4f} | Reg Penalty = {reg_penalty:,.0f} | ||θ|| = {np.linalg.norm(ridge_temp.coef_):.2f}")

print(f"\n✅ OPTIMAL RIDGE REGRESSION MODEL:")
print("-" * 40)
print(f"Best α (regularization): {best_alpha}")
print(f"Training R² score: {model.score(X_train_scaled, y_train):.4f}")
print(f"Number of features: {X_train_scaled.shape[1]}")
print(f"Bias term (intercept): {model.intercept_:.2f}")
print(f"First 5 feature weights: {model.coef_[:5]}")
print(f"Weight L2 norm: {np.linalg.norm(model.coef_):.2f}")

# Make predictions
train_predictions = model.predict(X_train_scaled)
test_predictions = model.predict(X_test_scaled)

# Compare with least regularized version
print(f"\n📊 REGULARIZATION IMPACT ANALYSIS:")
print("-" * 40)
unreg_results = all_results[0.01]  # Least regularized
reg_results = all_results[best_alpha]

print(f"Least regularized (α=0.01) weight norm: {unreg_results['coef_norm']:.2f}")
print(f"Optimal Ridge weight norm: {reg_results['coef_norm']:.2f}")
print(f"Weight reduction: {(1 - reg_results['coef_norm']/unreg_results['coef_norm'])*100:.1f}%")

# Store coefficients for analysis (including intercept)
weights = np.concatenate([[model.intercept_], model.coef_])


STEP 4: RIDGE REGRESSION - SCIKIT-LEARN IMPLEMENTATION
📐 Using sklearn Ridge Regression:
Objective: Minimize ||Xθ - y||² + λ||θ||²
Training data shape: (1200, 80)

🔍 HYPERPARAMETER TUNING - TESTING α VALUES:
--------------------------------------------------
✅ Best α (regularization) found via CV: 1000.0
α =     0.01 | R² = 0.8675 | Reg Penalty = 18,161,463 | ||θ|| = 42616.27
α =     0.10 | R² = 0.8675 | Reg Penalty = 181,521,771 | ||θ|| = 42605.37
α =     1.00 | R² = 0.8675 | Reg Penalty = 1,806,002,587 | ||θ|| = 42497.09
α =    10.00 | R² = 0.8675 | Reg Penalty = 17,204,224,708 | ||θ|| = 41477.98
α =   100.00 | R² = 0.8642 | Reg Penalty = 123,277,275,776 | ||θ|| = 35110.86
α =  1000.00 | R² = 0.8307 | Reg Penalty = 554,133,705,024 | ||θ|| = 23540.04

✅ OPTIMAL RIDGE REGRESSION MODEL:
----------------------------------------
Best α (regularization): 1000.0
Training R² score: 0.8307
Number of features: 80
Bias term (intercept): 181709.90
First 5 feature weights: [ -522.55500954 -1947.

In [28]:
# === STEP 5: ERROR FUNCTION ANALYSIS ===
# Following course concepts: Multiple Error Functions for Model Evaluation

print(f"\n" + "="*60)
print("STEP 5: ERROR FUNCTION ANALYSIS")
print("="*60)

def calculate_error_functions(y_true, y_pred):
    """
    Calculate multiple error functions as taught in course
    """
    n = len(y_true)
    
    # 1. Mean Squared Error (MSE) - L2 Loss
    mse = np.mean((y_true - y_pred) ** 2)
    
    # 2. Root Mean Squared Error (RMSE)
    rmse = np.sqrt(mse)
    
    # 3. Mean Absolute Error (MAE) - L1 Loss
    mae = np.mean(np.abs(y_true - y_pred))
    
    # 4. R-squared (Coefficient of Determination)
    ss_res = np.sum((y_true - y_pred) ** 2)  # Residual sum of squares
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)  # Total sum of squares
    r2 = 1 - (ss_res / ss_tot)
    
    # 5. Adjusted R-squared
    n_features = X_train_scaled.shape[1]
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - n_features - 1)
    
    # 6. Mean Absolute Percentage Error (MAPE)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    return {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R²': r2,
        'Adjusted R²': adj_r2,
        'MAPE': mape
    }

# Calculate error functions for training data
train_errors = calculate_error_functions(y_train, train_predictions)

print("📊 TRAINING SET ERROR ANALYSIS:")
print("-" * 40)
for metric, value in train_errors.items():
    if metric in ['MAPE']:
        print(f"{metric:12}: {value:.2f}%")
    elif metric in ['MSE']:
        print(f"{metric:12}: {value:,.0f}")
    elif metric in ['RMSE', 'MAE']:
        print(f"{metric:12}: ${value:,.2f}")
    else:
        print(f"{metric:12}: {value:.4f}")

# Analysis of prediction quality
print(f"\n📈 PREDICTION QUALITY ANALYSIS:")
print("-" * 40)
residuals = y_train - train_predictions
print(f"Mean residual: ${np.mean(residuals):,.2f}")
print(f"Std of residuals: ${np.std(residuals):,.2f}")
print(f"Min prediction: ${np.min(train_predictions):,.2f}")
print(f"Max prediction: ${np.max(train_predictions):,.2f}")
print(f"Predictions in range [0, max_actual]: {np.sum((train_predictions >= 0) & (train_predictions <= np.max(y_train))) / len(train_predictions) * 100:.1f}%")


STEP 5: ERROR FUNCTION ANALYSIS
📊 TRAINING SET ERROR ANALYSIS:
----------------------------------------
MSE         : 1,019,573,087
RMSE        : $31,930.75
MAE         : $19,068.46
R²          : 0.8307
Adjusted R² : 0.8186
MAPE        : 10.63%

📈 PREDICTION QUALITY ANALYSIS:
----------------------------------------
Mean residual: $-0.00
Std of residuals: $31,930.75
Min prediction: $46,436.98
Max prediction: $494,678.16
Predictions in range [0, max_actual]: 100.0%


In [29]:
# === STEP 6: MODEL DIAGNOSTICS AND VALIDATION ===
# Following course concepts: Model Validation and Diagnostics

print(f"\n" + "="*60)
print("STEP 6: MODEL DIAGNOSTICS AND VALIDATION")
print("="*60)

# 1. Prediction Examples (Sample Analysis)
print("🔍 PREDICTION EXAMPLES (First 10 samples):")
print("-" * 50)
comparison_df = pd.DataFrame({
    'Actual': y_train[:10],
    'Predicted': train_predictions[:10],
    'Residual': y_train[:10] - train_predictions[:10],
    'Abs_Error': np.abs(y_train[:10] - train_predictions[:10])
})
print(comparison_df.round(2))

# 2. Model Complexity Analysis
print(f"\n📊 MODEL COMPLEXITY ANALYSIS:")
print("-" * 40)
print(f"Number of training samples: {len(y_train)}")
print(f"Number of features: {X_train_scaled.shape[1]}")
print(f"Parameters to data ratio: {len(weights)}/{len(y_train)} = {len(weights)/len(y_train):.4f}")

# 3. Weight Analysis
print(f"\n⚖️ LEARNED PARAMETERS ANALYSIS:")
print("-" * 40)
print(f"Intercept (bias): {weights[0]:.4f}")
print(f"Largest positive weight: {np.max(weights[1:]):.4f}")
print(f"Largest negative weight: {np.min(weights[1:]):.4f}")
print(f"Weight standard deviation: {np.std(weights[1:]):.4f}")

# 4. Residual Analysis
print(f"\n📉 RESIDUAL ANALYSIS:")
print("-" * 40)
residuals = y_train - train_predictions
print(f"Residual mean (should be ~0): {np.mean(residuals):.4f}")
print(f"Residual std: {np.std(residuals):.2f}")
print(f"Residual skewness: {np.mean(((residuals - np.mean(residuals)) / np.std(residuals)) ** 3):.4f}")

# 5. Prediction Bounds Analysis
negative_predictions = np.sum(train_predictions < 0)
if negative_predictions > 0:
    print(f"\n⚠️ WARNING: {negative_predictions} negative predictions detected!")
    print("This suggests the model may need constraints or regularization.")

print(f"\n✅ MODEL VALIDATION COMPLETE!")


STEP 6: MODEL DIAGNOSTICS AND VALIDATION
🔍 PREDICTION EXAMPLES (First 10 samples):
--------------------------------------------------
     Actual  Predicted   Residual  Abs_Error
0  395000.0  284021.51  110978.49  110978.49
1  165000.0  188176.18  -23176.18   23176.18
2  128200.0  122934.61    5265.39    5265.39
3  275000.0  242525.45   32474.55   32474.55
4  311872.0  326352.84  -14480.84   14480.84
5  214000.0  240261.34  -26261.34   26261.34
6  153500.0  190146.54  -36646.54   36646.54
7  144000.0  158461.47  -14461.47   14461.47
8  115000.0  125563.42  -10563.42   10563.42
9  180000.0  180532.85    -532.85     532.85

📊 MODEL COMPLEXITY ANALYSIS:
----------------------------------------
Number of training samples: 1200
Number of features: 80
Parameters to data ratio: 81/1200 = 0.0675

⚖️ LEARNED PARAMETERS ANALYSIS:
----------------------------------------
Intercept (bias): 181709.8958
Largest positive weight: 8785.4688
Largest negative weight: -6014.6779
Weight standard deviation

In [30]:
# === STEP 7: MODEL ANALYSIS ===
# Analysis of the sklearn Ridge Regression model

print(f"\n" + "="*60)
print("STEP 7: RIDGE MODEL ANALYSIS")
print("="*60)

# 1. Ridge Regression Model Properties
print("📐 RIDGE REGRESSION MODEL PROPERTIES:")
print("-" * 50)
print(f"Optimal α (regularization): {model.alpha}")
print(f"Intercept: {model.intercept_:.4f}")
print(f"Number of coefficients: {len(model.coef_)}")
print(f"Coefficient L2 norm: {np.linalg.norm(model.coef_):.4f}")
print(f"Largest positive coefficient: {np.max(model.coef_):.4f}")
print(f"Largest negative coefficient: {np.min(model.coef_):.4f}")
print(f"Coefficient standard deviation: {np.std(model.coef_):.4f}")

# 2. Model Performance Metrics
print(f"\n📊 MODEL PERFORMANCE:")
print("-" * 50)
train_r2 = model.score(X_train_scaled, y_train)
print(f"Training R² score: {train_r2:.4f}")
print(f"Training R² percentage: {train_r2*100:.2f}%")

# Cross-validation score
cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='r2')
print(f"Cross-validation R² mean: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")

# 3. Regularization Effect Analysis
print(f"\n🎯 REGULARIZATION EFFECT ANALYSIS:")
print("-" * 50)
print("α Value | R² Score | Coef Norm | Reg Penalty")
print("-" * 45)
for alpha in sorted(all_results.keys()):
    result = all_results[alpha]
    print(f"{alpha:7.2f} | {result['r2']:8.4f} | {result['coef_norm']:9.2f} | {result['reg_penalty']:11,.0f}")

# 4. Feature Importance (absolute coefficient values)
print(f"\n🏆 TOP 10 MOST IMPORTANT FEATURES:")
print("-" * 50)
if hasattr(X_train_processed, 'columns'):
    feature_names = X_train_processed.columns
    feature_importance = pd.DataFrame({
        'Feature': feature_names,
        'Coefficient': model.coef_,
        'Abs_Coefficient': np.abs(model.coef_)
    }).sort_values('Abs_Coefficient', ascending=False)
    
    print(feature_importance.head(10).to_string(index=False))
else:
    # If we don't have feature names, show indices
    coef_indices = np.argsort(np.abs(model.coef_))[::-1][:10]
    for i, idx in enumerate(coef_indices):
        print(f"Feature {idx:2d}: {model.coef_[idx]:8.4f}")

# 5. Model Complexity Analysis
print(f"\n🔢 MODEL COMPLEXITY:")
print("-" * 50)
n_samples = len(y_train)
n_params = len(model.coef_) + 1  # +1 for intercept
df_residual = n_samples - n_params

print(f"Number of samples: {n_samples}")
print(f"Number of parameters: {n_params}")
print(f"Degrees of freedom (residual): {df_residual}")
print(f"Parameter/Sample ratio: {n_params/n_samples:.3f}")

# 6. Regularization Strength Assessment
print(f"\n⚖️ REGULARIZATION ASSESSMENT:")
print("-" * 50)
reg_penalty = model.alpha * np.sum(model.coef_ ** 2)
print(f"Regularization penalty: {reg_penalty:,.0f}")
print(f"Data fit term (MSE): {np.mean((y_train - train_predictions) ** 2):,.0f}")
print(f"Total objective value: {np.mean((y_train - train_predictions) ** 2) + reg_penalty:,.0f}")

if model.alpha < 1.0:
    print("✅ Light regularization - preserving model flexibility")
elif model.alpha < 100.0:
    print("⚖️ Moderate regularization - good bias-variance balance")
else:
    print("🔒 Strong regularization - emphasizing generalization")


STEP 7: RIDGE MODEL ANALYSIS
📐 RIDGE REGRESSION MODEL PROPERTIES:
--------------------------------------------------
Optimal α (regularization): 1000.0
Intercept: 181709.8958
Number of coefficients: 80
Coefficient L2 norm: 23540.0447
Largest positive coefficient: 8785.4688
Largest negative coefficient: -6014.6779
Coefficient standard deviation: 2529.2406

📊 MODEL PERFORMANCE:
--------------------------------------------------
Training R² score: 0.8307
Training R² percentage: 83.07%
Cross-validation R² mean: 0.7940 ± 0.0549

🎯 REGULARIZATION EFFECT ANALYSIS:
--------------------------------------------------
α Value | R² Score | Coef Norm | Reg Penalty
---------------------------------------------
   0.01 |   0.8675 |  42616.27 |  18,161,463
   0.10 |   0.8675 |  42605.37 | 181,521,771
   1.00 |   0.8675 |  42497.09 | 1,806,002,587
  10.00 |   0.8675 |  41477.98 | 17,204,224,708
 100.00 |   0.8642 |  35110.86 | 123,277,275,776
1000.00 |   0.8307 |  23540.04 | 554,133,705,024

🏆 TOP 10 

In [31]:
# === STEP 8: KAGGLE SUBMISSION PREPARATION ===
# Final step: Prepare submission file for Kaggle competition

print(f"\n" + "="*60)
print("STEP 8: KAGGLE SUBMISSION PREPARATION")
print("="*60)

# Load test data and make predictions
print("📁 LOADING TEST DATA AND MAKING PREDICTIONS:")
print("-" * 50)

# Test data was already preprocessed and scaled in Step 3
print(f"Test data shape: {X_test_scaled.shape}")

# Make predictions using our trained sklearn Ridge model
test_predictions = model.predict(X_test_scaled)

print(f"✅ Test predictions generated for {len(test_predictions)} samples")
print(f"Prediction range: [{test_predictions.min():.2f}, {test_predictions.max():.2f}]")

# Create submission file using actual test IDs
submission_df = pd.DataFrame({
    'Id': test_df['Id'].values,  # Use actual IDs from test dataset
    'HotelValue': test_predictions  # Use HotelValue as per sample submission format
})

# Save submission file
submission_path = '/Users/hemanthmada/vscodeProjects/ml_assignment_1/submissions/2_ridge_regression_sklearn.csv'
submission_df.to_csv(submission_path, index=False)

print(f"📤 SUBMISSION FILE CREATED:")
print("-" * 50)
print(f"File saved at: {submission_path}")
print(f"Submission shape: {submission_df.shape}")
print("\nFirst 5 predictions:")
print(submission_df.head())

# Final summary of the entire Ridge regression implementation
print(f"\n" + "="*60)
print("🎓 SKLEARN RIDGE REGRESSION SUMMARY")
print("="*60)
print("✅ Step 1: Data Loading and Exploration - COMPLETED")
print("✅ Step 2: Data Preprocessing and Feature Engineering - COMPLETED")
print("✅ Step 3: Feature Scaling - COMPLETED")
print("✅ Step 4: Sklearn Ridge Regression with CV - COMPLETED")
print("✅ Step 5: Comprehensive Error Function Analysis - COMPLETED")
print("✅ Step 6: Model Diagnostics and Validation - COMPLETED")
print("✅ Step 7: Ridge Model Analysis - COMPLETED")
print("✅ Step 8: Kaggle Submission Preparation - COMPLETED")
print(f"\n🏆 SKLEARN RIDGE REGRESSION MODEL READY!")
print(f"🎯 Optimal α: {model.alpha} | Training R²: {model.score(X_train_scaled, y_train):.4f}")
print(f"📊 Cross-validation R²: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")


STEP 8: KAGGLE SUBMISSION PREPARATION
📁 LOADING TEST DATA AND MAKING PREDICTIONS:
--------------------------------------------------
Test data shape: (260, 80)
✅ Test predictions generated for 260 samples
Prediction range: [52475.72, 460542.59]
📤 SUBMISSION FILE CREATED:
--------------------------------------------------
File saved at: /Users/hemanthmada/vscodeProjects/ml_assignment_1/submissions/2_ridge_regression_sklearn.csv
Submission shape: (260, 2)

First 5 predictions:
     Id     HotelValue
0   893  144645.784628
1  1106  302298.314606
2   414  111196.279900
3   523  163626.201948
4  1037  306632.375271

🎓 SKLEARN RIDGE REGRESSION SUMMARY
✅ Step 1: Data Loading and Exploration - COMPLETED
✅ Step 2: Data Preprocessing and Feature Engineering - COMPLETED
✅ Step 3: Feature Scaling - COMPLETED
✅ Step 4: Sklearn Ridge Regression with CV - COMPLETED
✅ Step 5: Comprehensive Error Function Analysis - COMPLETED
✅ Step 6: Model Diagnostics and Validation - COMPLETED
✅ Step 7: Ridge Model